In [1]:
# Test within subject variability 

# s1 ... s50 listen to the same story 
# Does f(s1) generalize better to f(s1) or f(s2). delta is the distance  

In [3]:
cd ..

/private/home/ccaucheteux/hasson-syntaxe-vs-semantics


In [8]:
def correlate(X, Y):
    if X.ndim == 1:
        X = X[:, None]
    if Y.ndim == 1:
        Y = Y[:, None]
    X = X - X.mean(0)
    Y = Y - Y.mean(0)
    SX2 = (X ** 2).sum(0) ** 0.5
    SY2 = (Y ** 2).sum(0) ** 0.5
    SXY = (X * Y).sum(0)
    return SXY / (SX2 * SY2)

In [77]:
KFold(5, shuffle=False)

5

In [88]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
import numpy as np
def encode(X, Y1, Y2, 
           model=RidgeCV(np.logspace(-2, 8, 10)), 
           cv = KFold(5, shuffle=False)):
    assert Y1.shape == Y2.shape
    
    r = np.zeros((cv.get_n_splits(), 2, 2, *Y1.shape[1:]))
    for i, (train, test) in enumerate(cv.split(X)):
        for j, Y in enumerate([Y1, Y2]):
            model.fit(X[train], Y[train])
            pred = model.predict(X[test])
            #import pdb
            #pdb.set_trace()
            r[i, j, 0] = correlate(Y1[test], pred)
            r[i, j, 1] = correlate(Y2[test], pred)
    return r        

In [64]:
from pathlib import Path

import numpy as np
from encoding.fmri import convolve_features
from nilearn import signal
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler, StandardScaler

from src import paths
from src.encode import encode_with_folds
from src.get_bold import get_bold
from src.get_features import load_precomputed_features
from src.preprocess_stim import add_pulses_to_stim, get_stimulus
from src.task_dataset import get_task_df

In [28]:
import numpy as np
from tqdm.notebook import tqdm

from src import paths
from src.get_bold import get_bold
from src.task_dataset import get_task_df
from pathlib import Path

def get_brain(hemi, df_task, task, save_path, space = "fsaverage6", 
             scaler = RobustScaler(quantile_range=(0.1, 99.9))):
    print(task, hemi)
    bold = []
    errors = []
    subs = []
    df_task = df_task.query("audio_task==@task")
    print(len(df_task))
    for i, row in tqdm(df_task.iterrows()):
        if i%100==0:
            print("i", i)
        # Load bold responses
        gii_fname = f"{row.subject}_task-{row.bold_task}_*space-{space}_hemi-{hemi}_desc-clean.func.gii"
        try:
            subj_data = get_bold(
            gii_fname, row.subject, exclude=True, afni_dir=paths.afni_dir_nosmooth
        )
        except AssertionError as e:
            print("Assertion error")
            errors.append(gii_fname)
            continue
        if subj_data is None:
            errors.append(gii_fname)
            continue        
        subj_data = scaler.fit_transform(subj_data[row.onset:])
        bold.append(subj_data)
        subs.append(row.subject)
    if len(bold)==0:
        med_bolds = {"bold": None, "count":0, "errors":errors}
    else:
        nr, nc = np.stack([b.shape for b in bold]).min(0)
        print(f"Cutting to {nr}, {nc}")
        count = len(bold)
        bold = [b[:nr, :nc] for b in bold]
        #bold = np.median(np.stack(bold), axis=0)
        med_bolds = {"bold": np.stack(bold), "count":count, "task":task, "errors":errors, "subjects":subs} 
    return med_bolds
    
    #Path(save_path).parent.mkdir(exist_ok=True, parents=True)
    #np.save(save_path, med_bolds)
    
def get_feats(feature_files, audio_task, word_idx, convolve_model="fir", n_delays=4, 
             scaler = StandardScaler()):
    word_idx = word_events.word_index.values

    # Extract features from stimulus
    feature_task_files = [Path(str(f) % str(audio_task)) for f in feature_files]
    assert np.all(
        [f.is_file() for f in feature_task_files]
    ), f"!!!!!! NOT EXIXTS : {feature_task_files}"
    features = load_precomputed_features(
        audio_task, feature_task_files, idx=word_idx
    )

    assert np.all([len(feat) == len(word_idx) for feat in features])
    
    convolved = []
    for feat in features:
        feat = scaler.fit_transform(feat)
        feat = convolve_features(
        events, feat, model=convolve_model, n_delays=n_delays
    )
        feat = scaler.fit_transform(feat)
        convolved.append(feat)
    
    return convolved

In [26]:
df_task = get_task_df()
#get_brain(hemi, df_task, task, save_path, space = "fsaverage6")

In [29]:
test = get_brain("L", df_task, "shapessocial", None, space = "fsaverage6")

shapessocial L
59


0it [00:00, ?it/s]

Excluding sub-238_task-shapessocial_space-fsaverage6_hemi-L_desc-clean.func.gii!
Cutting to 306, 40962


In [30]:
test["bold"].shape

(58, 306, 40962)

In [67]:
audio_task = "shapessocial"
trim_init = 2

In [70]:
TR = 1.5
subj_data = test["bold"].copy()
n_pulse = subj_data.shape[1]


stimuli = get_stimulus(audio_task)
events = add_pulses_to_stim(stimuli, n_pulse, TR=TR)
onset = np.floor(stimuli.dropna(subset=["onset"]).iloc[0].onset / TR)
onset = int(np.max([trim_init, onset]))
offset = np.ceil(stimuli.dropna(subset=["offset"]).iloc[-1].offset / TR)
offset = int(offset)

# Filter subj data
subj_data = subj_data[:, onset:offset]

events = events.query("volume<@offset and volume>=@onset")
events["volume"] -= onset
word_events = events.query("condition == 'word'")


word_idx = word_events.word_index.values
features = get_feats(feature_files, audio_task, word_idx)

In [74]:
subj_data.shape

(58, 273, 40962)

In [92]:
list(scores.keys())[0]

PosixPath('/private/home/ccaucheteux/hasson-syntaxe-vs-semantics/data/embeddings/0206_wordembed/%s/3_phone_features.pth')

In [100]:
k0 = Path('/private/home/ccaucheteux/hasson-syntaxe-vs-semantics/data/embeddings/0206_wordembed/%s/3_phone_features.pth')
scores[k0][0, 1, 0]


array([ 0.01298332, -0.01403668, -0.03671308, ..., -0.11719131,
       -0.11090633, -0.09700501])

In [101]:
len(subj_data)

58

In [106]:
3+3

6

In [108]:
scores.keys()

dict_keys([PosixPath('/private/home/ccaucheteux/hasson-syntaxe-vs-semantics/data/embeddings/0206_wordembed/%s/3_phone_features.pth'), PosixPath('/private/home/ccaucheteux/hasson-syntaxe-vs-semantics/data/embeddings/0206_wordembed/%s/sum-gpt2-0.pth')])

In [110]:
scores[Path('/private/home/ccaucheteux/hasson-syntaxe-vs-semantics/data/embeddings/0206_wordembed/%s/sum-gpt2-0.pth')].shape

(5, 2, 2, 40962)

In [ ]:
scores = {}
cv = KFold(5, shuffle=False)
for lab, feat in zip(feature_files, features):
    scores[lab] = np.zeros((len(subj_data), len(subj_data), 5, 2, 2, 40962))
    for i in range(len(subj_data)):
        for j in np.arange(i+1, len(subj_data)):
            print(i, j)
            scores[lab][i, j] = encode(feat, subj_data[i], subj_data[j], 
                   model=RidgeCV(np.logspace(1, 8, 8)), 
                   cv=cv)

In [33]:
FEATURE_FOLDER = "0206_wordembed"
feature_files = [paths.embeddings / FEATURE_FOLDER / "%s" / f"{feat}.pth" for feat in ["3_phone_features", 
                                                                                      "sum-gpt2-0",
                                                                                      "sum-gpt2-9"]]

In [ ]:
stimuli = get_stimulus(audio_task)

In [ ]:
for i in range()

In [103]:
df_task

,audio_task,subject,comprehension,bold_task,onset
0,pieman,sub-001,NaN,pieman,0
1,pieman,sub-002,NaN,pieman,0
2,pieman,sub-003,NaN,pieman,0
3,pieman,sub-004,NaN,pieman,0
4,pieman,sub-005,NaN,pieman,0
...,...,...,...,...,...
633,black,sub-311,0.66,black,8
634,black,sub-312,0.84,black,8
635,black,sub-313,0.88,black,8
636,black,sub-314,0.92,black,8


In [102]:
df_task.groupby("audio_task")["subject", "duration"].agg(["nunique", "first"])

<ipython-input-102-51316cedfd55>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_task.groupby("audio_task")["subject", "duration"].agg(["nunique", "first"])


KeyError: "Columns not found: 'duration'"

In [ ]:
df_task = get_task_df()
    df_task = df_task.query("subject==@subject and audio_task==@audio_task")
    print(df_task)
    row = df_task.iloc[0]
    audio_onset = row.onset
    audio_task = row.audio_task
    bold_task = row.bold_task

    r = {}
    features = None
    for hemi in hemis:
        # Load bold responses
        gii_fname = (
            f"{subject}_task-{bold_task}_*space-{space}_hemi-{hemi}_desc-clean.func.gii"
        )
        if smooth:
            afni_dir = paths.afni_dir
        else:
            afni_dir = paths.afni_dir_nosmooth
        subj_data = get_bold(gii_fname, subject, exclude=True, afni_dir=afni_dir)
        if subj_data is None:
            return

        # Trim bold w.r.t onset timing of the audio file => everything starts at 0 from now on
        subj_data = subj_data[audio_onset:, :]

        # Load stimulus
        stimuli = get_stimulus(audio_task)

        # Merge pulses and stimulus
        n_pulse = len(subj_data)
        events = add_pulses_to_stim(stimuli, n_pulse, TR=TR)

        # Cut extra pulses
        onset = np.floor(stimuli.dropna(subset=["onset"]).iloc[0].onset / TR)
        onset = int(np.max([trim_init, onset]))
        offset = np.ceil(stimuli.dropna(subset=["offset"]).iloc[-1].offset / TR)
        offset = int(offset)

        subj_data = subj_data[onset:offset]
        events = events.query("volume<@offset and volume>=@onset")
        events["volume"] -= onset
        assert len(subj_data) == len(events.query("condition=='Pulse'"))

        word_events = events.query("condition == 'word'")
        word_idx = word_events.word_index.values

        # Extract features from stimulus
        feature_task_files = [Path(str(f) % str(audio_task)) for f in feature_files]
        assert np.all(
            [f.is_file() for f in feature_task_files]
        ), f"!!!!!! NOT EXIXTS : {feature_task_files}"
        features = load_precomputed_features(
            audio_task, feature_task_files, idx=word_idx
        )

        assert np.all([len(feat) == len(word_events) for feat in features])

In [ ]:
def run_exp_single_tasks(
    subject,
    audio_task,
    feature_files=[],  # ["wordpos", "seqlen", "gpt2.0"],
    hemis=["L", "R"],
    smooth=False,
    space="fsaverage6",
    TR=1.5,
    trim_init=2,
    use_torch=False,
    convolve_model="fir",
    high_pass=None,
    n_folds=5,
    zero_out=None,
    regress_out=None,
    fit_intercept=False,
    other_task=False,
    average_folds=True,
    n_delays=4,
    scaler=RobustScaler(quantile_range=(0.1, 99.9)),
):

    

        # ENCODE HERE 

        r[hemi] = {}
        for lab, feat in zip(feature_files, features):
            feat = scaler.fit_transform(feat)
            convolved = convolve_features(
                events, feat, model=convolve_model, n_delays=n_delays
            )
            convolved = scaler.fit_transform(convolved)

            # Encode
            bold = subj_data.copy()
            valid = bold.std(0) > 0
            bold[:, valid] = scaler.fit_transform(bold[:, valid])

            # CONFOUNDS
            if zero_out is not None and zero_out[lab] is not None:
                to_zero_out = list(zero_out[lab]) * n_delays
                to_zero_out = np.array(to_zero_out)
            else:
                to_zero_out = None

            if regress_out is not None and regress_out[lab] is not None:
                to_regress_out = list(regress_out[lab]) * n_delays
                to_regress_out = np.array(to_regress_out)
            else:
                to_regress_out = None

            if average_folds:
                r[hemi][lab] = np.zeros(bold.shape[1])
            else:
                r[hemi][lab] = np.zeros((bold.shape[1], n_folds))
            r[hemi][lab][valid] = encode_with_folds(
                convolved,
                bold[:, valid],
                n_folds=n_folds,
                average_folds=average_folds,
                estimator=RidgeCV(
                    np.logspace(-1, 8, 10),
                    fit_intercept=fit_intercept,
                ),
                to_regress_out=to_regress_out,
                to_zero_out=to_zero_out,
                groups=None,
            )

    return r

In [ ]:
def encode(X1, X2, Y1, Y2):
    
    r1 = np.zeros(*Y1.shape[1:])
    r2 = np.zeros(*Y1.shape[1:])
    for train, test in cv.split(X1):
        for Xa, Ya, Xb, Yb in zip([X1, X2], 
                                  [Y1, Y2],
                                  [X2, X1],
                                  [Y2, Y1]):
            model.fit(Xa[train], Ya[train])
            preda = model.predict(Xa[test])
            predb = model.predict(Xb[test])

        model.fit(X2[train], )
        

In [ ]:
def encode(X, Y):
    """
    X of shape n_samples, n_conditions, n_features
    Y same
    """
    
    r = np.zeros(*X.shape[1:])
    for train, test in cv.split(X):
        for 
        model.fit(X[i, train], Y[i, train])
        pred[i] = model.predict(X1[test])

        model.fit(X2[train], )
        